In [3]:
pip install transformers torch

  Using cached transformers-4.51.2-py3-none-any.whl.metadata (38 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached transformers-4.51.2-py3-none-any.whl (10.4 MB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.8/204.1 MB 5.6 MB/s eta 0:00:37
   ---------------------------------------- 2.4/204.1 MB 6.4 MB/s eta 0:00:32
    --------------------------------------- 2.9/204.1 MB 4.9 MB/s eta 0:00:41
    --------------------------------------- 4.5/204.1 MB 5.5 MB/s eta 0:00:37
   - -------------------------------------- 5.5/204.1 MB 5.5 MB/s eta 0:00:37
   - -------------------------------------- 6.8/204.1 MB

In [5]:
import json
from transformers import pipeline
from pathlib import Path

In [ ]:
with open(r"C:\AISSMS-IOIT-Practicals\MLOps\data\fabry_disease.json", "r") as f:
    abstracts = json.load(f)

# Load local HF model
extractor = pipeline("text2text-generation", model="google/flan-t5-base")

triples = []

# Process each abstract
for item in abstracts:
    disease = "Fabry Disease"  # Can be inferred or hardcoded for now
    abstract = item["abstract"]

    prompt = f"""
    Extract knowledge triples from the following abstract in the format:
    [Disease] – has_symptom – [Symptom]
    [Disease] – treated_by – [Treatment]
    [Disease] - caused_by - [Cause]

    Abstract: {abstract}

    Output format:
    [
      {{ "subject": ..., "relation": ..., "object": ... }}
    ]
    """

    result = extractor(prompt, max_new_tokens=200)[0]["generated_text"]

    try:
        triples.extend(json.loads(result))
    except Exception as e:
        print(f"⚠️ Skipping malformed result for PMID {item['pmid']}:\n{result}\n")

# Save to JSON
Path("output").mkdir(exist_ok=True)
with open(r"C:\AISSMS-IOIT-Practicals\MLOps\data\triples.json", "w") as f:
    json.dump(triples, f, indent=2)

print("✅ Triples extracted and saved to triples.json")

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (700 > 512). Running this sequence through the model will result in indexing errors


⚠️ Skipping malformed result for PMID 40136155:
Fabry disease (FD) is an X-linked genetic condition caused by variants in the GLA gene causing enzyme -galactosidase A deficiency and accumulation of globotriaosylceramide (Gb3) in tissues such as the heart, kidneys, and the nervous system. This study reports a case series of patients with FD, highlighting the phenotypic diversity of the disease, which can be confused with other cardiological conditions. Early diagnosis of FD is fundamental for initiating treatments that can slow disease progression and prevent serious complications, reinforcing the need for greater awareness about this condition among cardiologists.

